In [1]:
#import tensorflow as tf
import torch
import numpy as np
from IPython.core.debugger import set_trace

class Graph():
    def __init__(self, num_node):
        self.num_node = num_node
        self.AD, self.AD2, self.bias_mat_1, self.bias_mat_2 = self.normalize_adjacency()
        
    def normalize_adjacency(self):
        self_link = [(i, i) for i in range(self.num_node)]
        neighbor_1base = [(0, 1), (1, 2), (2, 3), (3, 7), (0, 4), (4, 5),
                              (5, 6), (6, 8), (9, 10), (11, 12), (11, 13),
                              (13, 15), (15, 17), (15, 19), (15, 21), (17, 19),
                              (12, 14), (14, 16), (16, 18), (16, 20), (16, 22),
                              (18, 20), (11, 23), (12, 24), (23, 24), (23, 25),
                              (24, 26), (25, 27), (26, 28), (27, 29), (28, 30),
                              (29, 31), (30, 32), (27, 31), (28, 32)]
        neighbor_link = [(i - 1, j - 1) for (i, j) in neighbor_1base]
        edge = self_link + neighbor_link    
        A = np.zeros((self.num_node, self.num_node)) # adjacency matrix
        for i, j in edge:
            A[j, i] = 1
            A[i, j] = 1
        
        A2 = np.zeros((self.num_node, self.num_node)) # second order adjacency matrix
        for root in range(A.shape[1]):
            for neighbour in range(A.shape[0]):
                if A[root, neighbour] == 1:
                    for neighbour_of_neigbour in range(A.shape[0]):
                        if A[neighbour, neighbour_of_neigbour] == 1:
                            A2[root,neighbour_of_neigbour] = 1                 
        #AD = self.normalize(A)
        #AD2 = self.normalize(A2)
        bias_mat_1 = np.zeros(A.shape)
        bias_mat_2 = np.zeros(A2.shape)
        bias_mat_1 = np.where(A!=0, bias_mat_1, -1e9)
        bias_mat_2 = np.where(A2!=0, A2, -1e9)
        AD = A.astype('float32')
        AD2 = A2.astype('float32')
        bias_mat_1 = bias_mat_1.astype('float32')
        bias_mat_2 = bias_mat_2.astype('float32')
        #AD = tf.convert_to_tensor(AD)
        #AD2= tf.convert_to_tensor(AD2)
        AD=torch.tensor(AD).to('cuda:0')
        AD2=torch.tensor(AD2).to('cuda:0')
        #bias_mat_1 = tf.convert_to_tensor(bias_mat_1)
        #bias_mat_2 = tf.convert_to_tensor(bias_mat_2)
        bias_mat_1=torch.tensor(bias_mat_1).to('cuda:0')
        bias_mat_2=torch.tensor(bias_mat_2).to('cuda:0')
        return AD, AD2, bias_mat_1, bias_mat_2
        
    def normalize(self, adjacency):
        rowsum = np.array(adjacency.sum(1))
        r_inv = np.power(rowsum, -1).flatten()
        r_inv[np.isinf(r_inv)] = 0
        r_mat_inv = np.diag(r_inv)
        normalize_adj = r_mat_inv.dot(adjacency)
        normalize_adj = normalize_adj.astype('float32')
        #normalize_adj = tf.convert_to_tensor(normalize_adj)
        normalize_adj=torch.tensor(normalize_adj)   
        return normalize_adj

In [35]:
import torch.nn as nn
import torch
import torch.nn.functional as F





class sgcn(nn.Module):
    def __init__(self, AD, AD2,bias_mat_1, bias_mat_2, batch_size=10):
        super(sgcn, self).__init__()
        self.AD = AD
        self.AD2 = AD2
        self.batch_size = batch_size
        self.num_joints = 33
        self.bias_mat_1 = bias_mat_1
        self.bias_mat_2 = bias_mat_2
        self.bias_mat_1.to('cuda:0')
        self.bias_mat_2.to('cuda:0')



        #self.conv_lstm1=ConvLSTM(64,33,(1,1),1,True, True, False)
        #self.conv_lstm2=ConvLSTM(64,33,(1,1),1,True, True, False)
        self.dropout = nn.Dropout2d(p=0.25)
        #self.bias_mat_1=torch.zeros((33,33)).to('cuda:0')
        #self.bias_mat_2=torch.zeros((33,33)).to('cuda:0')
        self.conv1=nn.Conv2d(3, 64, kernel_size=(9, 1), padding=(4, 0)).to('cuda:0')
        self.conv2=nn.Conv2d(67, 64, kernel_size=(1, 1)).to('cuda:0')
        self.conv3=nn.Conv2d(67, 64, kernel_size=(1, 1)).to('cuda:0')
        self.conv4=nn.Conv2d(128, 16, kernel_size=(9, 1), padding=(4, 0),).to('cuda:0')
        self.conv5=nn.Conv2d(16, 16, kernel_size=(15, 1), padding=(7, 0)).to('cuda:0')
        self.conv6=nn.Conv2d(16, 16, kernel_size=(19, 1), padding=(9, 0)).to('cuda:0')
        self.relu=F.relu


    def forward(self, Input):
        print('Input:', Input.shape)
        #k1 = F.relu(nn.Conv2d(Input.shape[1], 64, kernel_size=(9, 1), padding=(4, 0)).to('cuda:0')(Input))
        k1 = F.relu(self.conv1(Input))
        print('k1:',k1.shape)

        k = torch.cat((Input, k1), dim=1)
        print('k:',k.shape)

        #x1 = F.relu(nn.Conv2d(Input.shape[1] + k1.shape[1], 64, kernel_size=(1, 1)).to('cuda:0')(k))
        x1 = self.relu(self.conv2(k))
        
        gcn_x1 = torch.einsum('vw,ncwt->ncvt', (self.AD, x1))


        #y1 = F.relu(nn.Conv2d(k.shape[1], 64, kernel_size=(1, 1)).to('cuda:0')(k))
        y1 = F.relu(self.conv3(k))
        gcn_y1 = torch.einsum('vw,ncwt->ncvt', (self.AD2, y1))

        gcn_1 = torch.cat((gcn_x1, gcn_y1), dim=1)
        print('gcn_1:',gcn_1.shape)

        #z1 = F.relu(nn.Conv2d(gcn_1.shape[1], 16, kernel_size=(9, 1), padding=(4, 0)).to('cuda:0')(gcn_1))
        z1 = F.relu(self.conv4(gcn_1))
        z1 = self.dropout(z1)

        print('z1:',z1.shape)
        #z2 = F.relu(nn.Conv2d(z1.shape[1], 16, kernel_size=(15, 1), padding=(7, 0)).to('cuda:0')(z1))
        z2 = F.relu(self.conv5(z1))
        z2 = self.dropout(z2)

        print('z2:',z2.shape)
        #z3 = F.relu(nn.Conv2d(z2.shape[1], 16, kernel_size=(19, 1), padding=(9, 0)).to('cuda:0')(z2))
        z3 = F.relu(self.conv6(z2))
        z3 = self.dropout(z3)

        z = torch.cat((z1, z2, z3), dim=1)

        return z

class Sgcn_Lstm(nn.Module):
    def __init__(self, train_x=None, train_y=None, AD=None, AD2=None, lr=0.0001, epoach=200, batch_size=10):
        super(Sgcn_Lstm, self).__init__()
        #self.train_x = train_x
        #self.train_y = train_y
        #self.AD = AD
        #self.AD2 = AD2
        #self.lr = lr
        #self.epoach =epoach
        #self.batch_size = batch_size
        #self.num_joints = 543
        graph=Graph(33)



        self.sgcn_1=sgcn(graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2)
        self.sgcn_2=sgcn(graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2)
        self.sgcn_3=sgcn(graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2)

    def lstm(self,x):
        x = x.view(x.shape[0], -1, x.shape[1] * x.shape[2])
        rec,_ = nn.LSTM(input_size=x.shape[2], hidden_size=256, num_layers=1, batch_first=True, dropout=0.25, bidirectional=False).to('cuda:0')(x)
        """
        rec,_ = nn.LSTM(input_size=x.shape[2], hidden_size=64, num_layers=1, batch_first=True, dropout=0.25, bidirectional=True).to('cuda:0')(x)
        rec1,_ = nn.LSTM(input_size=rec.shape[-1], hidden_size=32, num_layers=1, batch_first=True, dropout=0.25, bidirectional=True).to('cuda:0')(rec)
        rec2,_ = nn.LSTM(input_size=rec1.shape[-1], hidden_size=32, num_layers=1, batch_first=True, dropout=0.25, bidirectional=True).to('cuda:0')(rec1)
        rec3,_ = nn.LSTM(input_size=rec2.shape[-1], hidden_size=256, num_layers=1, batch_first=True, dropout=0.25, bidirectional=False).to('cuda:0')(rec2)
        """
        out=rec
        return out


    def forward(self, x):
        x = x.permute(0,3,2,1)
        x = self.sgcn_1(x)
        y = self.sgcn_2(x)
        y = y + x
        z = self.sgcn_3(y)
        z = z + y
        """
        batch_size, _, _, frame_size = z.size()
        z = z.view(batch_size * frame_size, -1)
        z=nn.Linear(48 * 33, 256).to('cuda:0')(z)
        z=z.view(batch_size,frame_size,-1)
        #print('z: ',z.shape)
        out=z
        """

        out = self.lstm(z)
        return out

In [36]:
import sacrebleu
from torchviz import make_dot
input=torch.randn(4, 60, 33, 3).to('cuda:0')
input=input.permute(0,3,2,1)
print(input.shape)
graph=Graph(33)
model=sgcn(graph.AD, graph.AD2, graph.bias_mat_1, graph.bias_mat_2)
#model=Sgcn_Lstm()
model = model.to('cuda:0')
#print(model)

# Perform a forward pass
output = model(input)

# Generate the block diagram
#dot=make_dot(output, params=dict(model.named_parameters()),show_saved=True)
#dot.save('/media/rmedu/Softwares/RabeyaAkter/slt_how2sign_wicv2023/fairseq/models/sign_to_text/sample_graph.dot')


torch.Size([4, 3, 33, 60])
Input: torch.Size([4, 3, 33, 60])
k1: torch.Size([4, 64, 33, 60])
k: torch.Size([4, 67, 33, 60])
gcn_1: torch.Size([4, 128, 33, 60])
z1: torch.Size([4, 16, 33, 60])
z2: torch.Size([4, 16, 33, 60])


In [12]:
from torchsummary import summary

In [37]:
summary(model,(3,33,60))

Input: torch.Size([2, 3, 33, 60])
k1: torch.Size([2, 64, 33, 60])
k: torch.Size([2, 67, 33, 60])
gcn_1: torch.Size([2, 128, 33, 60])
z1: torch.Size([2, 16, 33, 60])
z2: torch.Size([2, 16, 33, 60])
Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 33, 60]          1,792
├─Conv2d: 1-2                            [-1, 64, 33, 60]          4,352
├─Conv2d: 1-3                            [-1, 64, 33, 60]          4,352
├─Conv2d: 1-4                            [-1, 16, 33, 60]          18,448
├─Dropout2d: 1-5                         [-1, 16, 33, 60]          --
├─Conv2d: 1-6                            [-1, 16, 33, 60]          3,856
├─Dropout2d: 1-7                         [-1, 16, 33, 60]          --
├─Conv2d: 1-8                            [-1, 16, 33, 60]          4,880
├─Dropout2d: 1-9                         [-1, 16, 33, 60]          --
Total params: 37,680
Trainable params: 37,680
Non-trainable params: 0
Total mul

Layer (type:depth-idx)                   Output Shape              Param #
├─Conv2d: 1-1                            [-1, 64, 33, 60]          1,792
├─Conv2d: 1-2                            [-1, 64, 33, 60]          4,352
├─Conv2d: 1-3                            [-1, 64, 33, 60]          4,352
├─Conv2d: 1-4                            [-1, 16, 33, 60]          18,448
├─Dropout2d: 1-5                         [-1, 16, 33, 60]          --
├─Conv2d: 1-6                            [-1, 16, 33, 60]          3,856
├─Dropout2d: 1-7                         [-1, 16, 33, 60]          --
├─Conv2d: 1-8                            [-1, 16, 33, 60]          4,880
├─Dropout2d: 1-9                         [-1, 16, 33, 60]          --
Total params: 37,680
Trainable params: 37,680
Non-trainable params: 0
Total mult-adds (M): 74.13
Input size (MB): 0.02
Forward/backward pass size (MB): 3.63
Params size (MB): 0.14
Estimated Total Size (MB): 3.79

In [38]:
output.shape

torch.Size([4, 48, 33, 60])

In [2]:
import torch

# Assuming you have two tensors named tensor1 and tensor2
tensor1 = torch.randn(100, 32, 256)
tensor2 = torch.randn(200, 32, 256)

# Get the maximum sequence length
max_seq_len = max(tensor1.shape[0], tensor2.shape[0])

# Pad the tensors
tensor1_padded = torch.nn.functional.pad(tensor1, pad=(max_seq_len - tensor1.shape[0], 0, 0), mode='constant', value=0)
tensor2_padded = torch.nn.functional.pad(tensor2, pad=(max_seq_len - tensor2.shape[0], 0, 0), mode='constant', value=0)

# Add the tensors together
result_tensor = tensor1_padded + tensor2_padded

print(result_tensor.shape)

AssertionError: Padding length must be divisible by 2